In [1]:
import importlib.util
import math
import os.path
from os import path
import time

from fastai.script import *
from fastai.vision import *
from fastai.callbacks import *
from fastai.distributed import *
from fastai.callbacks.tracker import *
torch.backends.cudnn.benchmark = True
import numpy as np
import pandas as pd
from torch import nn

In [2]:
df = pd.read_csv('./labels_sample.csv')

In [3]:
df.head(5)

,image_name,tags
0,57284,4 6 27 54 70 49 23 10 3 3
1,57225,3 10 40 62 67 38 19 10 6 3
2,57217,1 2 13 29 78 68 36 18 4 4
3,57252,5 22 34 58 63 25 9 6 2 1
4,57211,3 12 35 57 81 41 16 6 5 1


In [ ]:
# df = df.set_index('image_name').T
# scores_map = df.to_dict('list')
# t = map((lambda pair: [pair[0], pair[1]]), zip(df.image_name, df.tags))

In [4]:
scores_map = dict(zip(df.image_name, df.tags))

In [ ]:
# scores = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
# scores = np.arange(1,11)

In [5]:
tfms = ([
    flip_lr(p=0.5),
    brightness(change=(0.4,0.6)),
    contrast(scale=(0.7,1.3))
], [])

In [6]:
labels = list(scores_map.keys())

## Examples

items = np.array([1, 2, 4, 5])
cl = CategoryList(items, classes=['1', '2', '4']); cl

items = np.array(['3', '7', '9'])
classes = {'3':3, '7':7, '9':9}
cl = CategoryList(items, classes); cl

items = np.array([3, 7, 9])
classes = {3:3, 7:7, 9:9}
cl = CategoryList(items, classes); cl

items = np.array(labels)
classes = dict(zip(labels, labels))
cl = CategoryList(items, classes); cl

In [7]:
# print(scores_map[int((o.split('/')[2]).split('.')[0])])
# func = lambda o: scores_map[int((o.split('/')[2]).split('.')[0])]
func = lambda o: int((o.split('/')[2]).split('.')[0])
# func = lambda o: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [8]:
class NimaLabelList(CategoryList):
    _processor=None
    def __init__(self, items:Iterator, classes=labels, label_delim:str=None, **kwargs):
        super().__init__(items, classes=classes, **kwargs)

    def get(self, i):
        dist = scores_map[self.items[i]]
        dist = np.array(dist.split(' '), dtype=float)
        return dist

In [10]:
data = (ImageList.from_csv('./', 'labels_sample.csv', folder='data', suffix='.jpg')
        .split_by_rand_pct()
        .label_from_func(func, label_cls=NimaLabelList)
        .transform(tfms, size=224)
        .databunch(bs=8))

In [11]:
data

ImageDataBunch;

Train: LabelList (60 items)
x: ImageList
Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224)
y: NimaLabelList
[ 4.  6. 27. 54. 70. 49. 23. 10.  3.  3.],[ 1.  2. 13. 29. 78. 68. 36. 18.  4.  4.],[ 5. 22. 34. 58. 63. 25.  9.  6.  2.  1.],[ 3. 12. 35. 57. 81. 41. 16.  6.  5.  1.],[ 1. 11. 35. 61. 61. 47. 21.  8.  0.  2.]
Path: .;

Valid: LabelList (14 items)
x: ImageList
Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224)
y: NimaLabelList
[ 2.  8. 15. 31. 62. 58. 35. 21. 12.  8.],[ 1.  4. 18. 39. 66. 56. 34. 23.  6.  5.],[13. 20. 52. 83. 47. 19.  7.  5.  1.  2.],[ 4. 15. 34. 59. 63. 44. 15.  3.  4.  3.],[ 3. 10. 40. 62. 67. 38. 19. 10.  6.  3.]
Path: .;

Test: None

In [12]:
# data.c, data.classes

In [13]:
data.c

74

In [14]:
data.c = 10

In [15]:
x,y = next(iter(data.train_dl))

In [16]:
len(x), len(y)

(8, 8)

In [17]:
#x[0]
y[0]

tensor([10., 21., 29., 44., 44., 41., 19., 15., 14.,  4.], device='cuda:0',
       dtype=torch.float64)

In [ ]:
# data.show_batch(rows=5, figsize=(9,7))

In [18]:
# learn = cnn_learner(data, models.resnet34, metrics=error_rate)
arch  = models.mobilenet_v2
learner = cnn_learner(data, arch, pretrained=True) #, loss_func=emd)

In [19]:
learner.model

Sequential(
  (0): Sequential(
    (0): Sequential(
      (0): ConvBNReLU(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
      )
      (1): InvertedResidual(
        (conv): Sequential(
          (0): ConvBNReLU(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU6(inplace=True)
          )
          (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (2): InvertedResidual(
        (conv): Sequential(
          (0): ConvBNReLU(
            (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
            

In [21]:
y_hat = learner.model(x)

In [22]:
y_hat

tensor([[-0.8907, -0.0284,  2.1365, -0.1709, -2.9459,  0.9273,  0.1555, -1.9065,
         -1.5195,  1.2261],
        [-0.8709, -1.6950,  0.5611, -1.0058,  4.5254,  0.0746, -0.5154, -0.4130,
          1.4756, -1.8157],
        [ 0.3191,  1.2512, -3.3986, -1.5026,  0.6233, -0.8968, -0.6463, -1.0154,
          0.9831, -1.1970],
        [ 1.4721, -3.9575, -0.8889, -2.6205,  0.5283, -1.4837,  1.1115,  1.6495,
         -4.7993, -0.5292],
        [ 1.9521,  7.6346, -2.4970,  0.9646, -0.6922, -2.5849, -1.9102,  2.3076,
          1.4259,  0.6265],
        [-5.6787,  4.1471, -1.9002, -2.0410, -2.3427,  1.5345, -1.8466,  2.0069,
         -2.3520, -0.6644],
        [ 0.2179, -1.2132,  1.0428,  3.6964,  1.0496, -0.5973, -0.7397,  0.6288,
         -2.7160,  4.5467],
        [-1.4032,  1.1219,  1.5479,  2.4881,  1.5284,  2.1760,  2.3377,  1.1735,
          1.5234,  0.9896]], device='cuda:0', grad_fn=<AddmmBackward>)

In [23]:
y_hat.shape, y.shape

(torch.Size([8, 10]), torch.Size([8, 10]))

In [24]:
y[0], torch.cumsum(y[0], dim=0)

(tensor([10., 21., 29., 44., 44., 41., 19., 15., 14.,  4.], device='cuda:0',
        dtype=torch.float64),
 tensor([ 10.,  31.,  60., 104., 148., 189., 208., 223., 237., 241.],
        device='cuda:0', dtype=torch.float64))

In [25]:
# (input, target)
def emd(y_hat, y):
    cdf_y = torch.cumsum(y, dim=-1)
    cdf_y_hat = torch.cumsum(y_hat, dim=-1).double()
    power = torch.pow((cdf_y - cdf_y_hat), 2)
    emd = torch.sqrt(torch.mean(power, dim=-1))
    return torch.mean(emd)

In [26]:
emd(y_hat, y)

tensor(180.6949, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)

In [27]:
learner

Learner(data=ImageDataBunch;

Train: LabelList (60 items)
x: ImageList
Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224)
y: NimaLabelList
[ 4.  6. 27. 54. 70. 49. 23. 10.  3.  3.],[ 1.  2. 13. 29. 78. 68. 36. 18.  4.  4.],[ 5. 22. 34. 58. 63. 25.  9.  6.  2.  1.],[ 3. 12. 35. 57. 81. 41. 16.  6.  5.  1.],[ 1. 11. 35. 61. 61. 47. 21.  8.  0.  2.]
Path: .;

Valid: LabelList (14 items)
x: ImageList
Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224)
y: NimaLabelList
[ 2.  8. 15. 31. 62. 58. 35. 21. 12.  8.],[ 1.  4. 18. 39. 66. 56. 34. 23.  6.  5.],[13. 20. 52. 83. 47. 19.  7.  5.  1.  2.],[ 4. 15. 34. 59. 63. 44. 15.  3.  4.  3.],[ 3. 10. 40. 62. 67. 38. 19. 10.  6.  3.]
Path: .;

Test: None, model=Sequential(
  (0): Sequential(
    (0): Sequential(
      (0): ConvBNReLU(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, ep

In [28]:
learner.loss_func

FlattenedLoss of CrossEntropyLoss()

In [29]:
learner.loss_func = emd

In [30]:
learner.loss_func

<function __main__.emd(y_hat, y)>

In [31]:
learner.fit_one_cycle(1,1e-3)

epoch,train_loss,valid_loss,time
0,172.083878,175.212563,00:01
